### Modelo Final Bayesiano - Ideal

In [ ]:
!pip install "pymc==5.25.1"
!pip install "pytensor>=2.31.7,<2.32"
!pip install arviz pandas numpy plotly streamlit
!pip install ipywidgets

In [16]:
"""
Script: treinar_modelo_bayesiano_final.py

Modelo bayesiano NEGATIVE BINOMIAL hierárquico para previsão de
ocorrências atendidas pela PMDF (2022–2024).

Saídas salvas em:
    data/bayes/modelofinal/idata_modelofinal.pkl
    data/bayes/modelofinal/posterior_summary.json
    data/bayes/modelofinal/predicoes_in_sample.json
    data/bayes/modelofinal/model_config.json
"""

import os
import json
import pickle
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az


# -----------------------------------------------------------------------------
# 1. Carregamento e preparação dos dados
# -----------------------------------------------------------------------------

CSV_PATH = "data/PMDF_ocorrencias_2022-2024.csv"
OUTPUT_DIR = "data/bayes/modelofinal"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)

# Mapeia mês em número
map_mes = {
    "JANEIRO": 1,
    "FEVEREIRO": 2,
    "MARÇO": 3,
    "MARCO": 3,
    "ABRIL": 4,
    "MAIO": 5,
    "JUNHO": 6,
    "JULHO": 7,
    "AGOSTO": 8,
    "SETEMBRO": 9,
    "OUTUBRO": 10,
    "NOVEMBRO": 11,
    "DEZEMBRO": 12,
}
df["mes_num"] = df["mes"].map(map_mes)

# Ordena temporalmente
df = df.sort_values(["ano", "mes_num"]).reset_index(drop=True)

# Índices hierárquicos
df["mes_idx"] = df["mes_num"] - 1
df["ano_idx"] = df["ano"].astype("category").cat.codes

y = df["ocor_atend"].astype("int64").values
n_obs = len(df)
n_mes = df["mes_idx"].nunique()
n_ano = df["ano_idx"].nunique()

mes_idx = df["mes_idx"].values
ano_idx = df["ano_idx"].values

# -----------------------------------------------------------------------------
# 2. Covariáveis (opcionais)
# -----------------------------------------------------------------------------

COVARIATE_CANDIDATES = [
    "arm_fogo_apr",
    "arm_branc_apr",
    "drog_kg_apr",
    "drog_un_apr",
]

covariate_cols = [c for c in COVARIATE_CANDIDATES if c in df.columns]

if covariate_cols:
    X = df[covariate_cols].astype(float).copy()
    X_mean = X.mean()
    X_std = X.std().replace(0, 1.0)
    X_z = (X - X_mean) / X_std
    X_matrix = X_z.values
    n_cov = X_matrix.shape[1]
else:
    X_matrix = None
    n_cov = 0
    X_mean = pd.Series(dtype=float)
    X_std = pd.Series(dtype=float)

covariate_metadata = {
    "covariate_cols": covariate_cols,
    "means": X_mean.to_dict(),
    "stds": X_std.to_dict(),
}

# -----------------------------------------------------------------------------
# 3. Modelo Bayesiano final (NegBin hierárquico sem RW)
# -----------------------------------------------------------------------------

coords = {
    "obs_id": np.arange(n_obs),
    "mes": np.arange(n_mes),
    "ano": np.arange(n_ano),
}
if n_cov > 0:
    coords["covariate"] = np.arange(n_cov)

with pm.Model(coords=coords) as modelo_final:

    # Dados (pm.Data é mutável no PyMC 5.25.1)
    mes_idx_data = pm.Data("mes_idx", mes_idx, dims="obs_id")
    ano_idx_data = pm.Data("ano_idx", ano_idx, dims="obs_id")
    if n_cov > 0:
        X_data = pm.Data("X", X_matrix, dims=("obs_id", "covariate"))

    # Intercepto global (no log da média)
    # 9.5–10 é da ordem de grandeza das ocorrências (exp(9.5) ~ 13k, exp(10) ~ 22k)
    alpha0 = pm.Normal("alpha0", mu=9.8, sigma=1.0)

    # Efeito aleatório de mês (sazonalidade)
    sigma_mes = pm.Exponential("sigma_mes", 2.0)
    mes_raw = pm.Normal("mes_raw", 0.0, 1.0, dims="mes")
    efeito_mes = pm.Deterministic("efeito_mes", mes_raw * sigma_mes, dims="mes")

    # Efeito aleatório de ano
    sigma_ano = pm.Exponential("sigma_ano", 2.0)
    ano_raw = pm.Normal("ano_raw", 0.0, 1.0, dims="ano")
    efeito_ano = pm.Deterministic("efeito_ano", ano_raw * sigma_ano, dims="ano")

    # Covariáveis padronizadas
    if n_cov > 0:
        beta = pm.Normal("beta", mu=0.0, sigma=0.5, dims="covariate")
        cov_effect = pm.math.dot(X_data, beta)
    else:
        beta = None
        cov_effect = 0.0

    # Preditor linear
    log_mu = alpha0 + efeito_mes[mes_idx_data] + efeito_ano[ano_idx_data] + cov_effect
    mu = pm.Deterministic("mu", pm.math.exp(log_mu), dims="obs_id")

    # Overdispersion da NegBin
    alpha_nb = pm.Exponential("alpha_nb", 1.0)

    # Likelihood
    y_obs = pm.NegativeBinomial(
        "y_obs",
        mu=mu,
        alpha=alpha_nb,
        observed=y,
        dims="obs_id",
    )

    # Amostragem – target_accept alto para reduzir divergências
    idata = pm.sample(
        draws=2000,
        tune=2000,
        chains=4,
        cores=4,
        target_accept=0.98,
        max_treedepth=12,
        random_seed=123,
    )

    # Posterior preditiva in-sample
    ppc = pm.sample_posterior_predictive(
        idata,
        var_names=["y_obs", "mu"],
        random_seed=123,
    )

idata.extend(ppc)

# -----------------------------------------------------------------------------
# 4. Resumos e objetos para o Dashboard
# -----------------------------------------------------------------------------

var_names_summary = ["alpha0", "sigma_mes", "sigma_ano", "alpha_nb"]
if n_cov > 0:
    var_names_summary.append("beta")

summary_df = az.summary(idata, var_names=var_names_summary)
summary_json = summary_df.reset_index().to_dict(orient="records")

# Predições in-sample (mediana + IC 95%)
y_pred_samples = idata.posterior_predictive["y_obs"].values  # (chain, draw, obs)
y_pred_flat = y_pred_samples.reshape(-1, n_obs)

y_pred_mediana = np.median(y_pred_flat, axis=0)
y_pred_hdi = az.hdi(y_pred_flat, hdi_prob=0.95)  # (obs, 2)

pred_df = df[["ano", "mes", "mes_num", "ocor_atend"]].copy()
pred_df["y_pred_mediana"] = y_pred_mediana
pred_df["y_pred_hdi_low"] = y_pred_hdi[:, 0]
pred_df["y_pred_hdi_high"] = y_pred_hdi[:, 1]

predicoes_json = pred_df.to_dict(orient="records")

model_config = {
    "family": "NegativeBinomial",
    "link": "log",
    "formula": (
        "ocor_atend ~ 1 + (1 | mes) + (1 | ano)"
        + (" + covariaveis_padronizadas" if n_cov > 0 else "")
    ),
    "priors": {
        "alpha0": "Normal(9.8, 1.0)",
        "sigma_mes": "Exponential(2.0)",
        "sigma_ano": "Exponential(2.0)",
        "alpha_nb": "Exponential(1.0)",
        "beta": "Normal(0, 0.5) em covariáveis z-score" if n_cov > 0 else None,
    },
    "covariate_metadata": covariate_metadata,
    "descricao": (
        "Modelo bayesiano hierárquico NegBin com efeitos aleatórios de mês "
        "e ano, e covariáveis de apreensões padronizadas (quando presentes)."
    ),
}

# -----------------------------------------------------------------------------
# 5. Salvando .pkl e .json
# -----------------------------------------------------------------------------

idata_path = os.path.join(OUTPUT_DIR, "idata_modelofinal.pkl")
with open(idata_path, "wb") as f:
    pickle.dump(idata, f)

summary_path = os.path.join(OUTPUT_DIR, "posterior_summary.json")
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary_json, f, ensure_ascii=False, indent=2)

pred_path = os.path.join(OUTPUT_DIR, "predicoes_in_sample.json")
with open(pred_path, "w", encoding="utf-8") as f:
    json.dump(predicoes_json, f, ensure_ascii=False, indent=2)

config_path = os.path.join(OUTPUT_DIR, "model_config.json")
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(model_config, f, ensure_ascii=False, indent=2)

print("Treinamento concluído.")
print(f"InferenceData salvo em: {idata_path}")
print(f"Resumo posterior salvo em: {summary_path}")
print(f"Predições in-sample salvas em: {pred_path}")
print(f"Configuração do modelo salva em: {config_path}")


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha0, sigma_mes, mes_raw, sigma_ano, ano_raw, beta, alpha_nb]


Output()

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 25 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
Sampling: [y_obs]


Output()

Treinamento concluído.
InferenceData salvo em: data/bayes/modelofinal/idata_modelofinal.pkl
Resumo posterior salvo em: data/bayes/modelofinal/posterior_summary.json
Predições in-sample salvas em: data/bayes/modelofinal/predicoes_in_sample.json
Configuração do modelo salva em: data/bayes/modelofinal/model_config.json


/tmp/ipykernel_53352/1954394299.py:192: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  y_pred_hdi = az.hdi(y_pred_flat, hdi_prob=0.95)  # (obs, 2)


### Predições para 2025

In [24]:
"""
Previsão 2025 – Versão Final Corrigida
"""

import os
import json
import pickle
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az


DATA_DIR = "data/bayes/modelofinal"
df = pd.read_csv("data/PMDF_ocorrencias_2022-2024.csv")

map_mes = {
    "JANEIRO": 1,"FEVEREIRO": 2,"MARÇO": 3,"MARCO": 3,"ABRIL": 4,"MAIO": 5,
    "JUNHO": 6,"JULHO": 7,"AGOSTO": 8,"SETEMBRO": 9,"OUTUBRO": 10,
    "NOVEMBRO": 11,"DEZEMBRO": 12
}

df["mes_num"] = df["mes"].map(map_mes)
df = df.sort_values(["ano", "mes_num"]).reset_index(drop=True)

df["mes_idx"] = df["mes_num"] - 1
df["ano_idx"] = df["ano"].astype("category").cat.codes

y = df["ocor_atend"].astype(int).values
mes_idx = df["mes_idx"].values.astype("int32")
ano_idx = df["ano_idx"].values.astype("int32")

n_obs = len(df)
n_mes = df["mes_idx"].nunique()

# ================================================================
# AQUI ESTÁ A CORREÇÃO DEFINITIVA:
# adicionar +1 ao número de anos para permitir ano_idx=3 (2025)
# ================================================================
n_ano_obs = df["ano_idx"].nunique()      # 3 anos observados
n_ano_total = n_ano_obs + 1              # adicionar ano 2025 → total=4

# -------------------------------------------------------
# Carregar metadata das covariáveis
# -------------------------------------------------------

with open(f"{DATA_DIR}/model_config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

cov_cols = config["covariate_metadata"]["covariate_cols"]
means = config["covariate_metadata"]["means"]
stds = config["covariate_metadata"]["stds"]

if cov_cols:
    X = df[cov_cols].copy().astype(float)
    X_z = (X - pd.Series(means)) / pd.Series(stds)
    X_matrix = X_z.values
    n_cov = X_matrix.shape[1]
else:
    X_matrix = None
    n_cov = 0

# -------------------------------------------------------
# Reconstruir o modelo com ano_total=4
# -------------------------------------------------------

coords = {
    "obs_id": np.arange(n_obs),
    "t_pred": np.arange(12),
    "mes": np.arange(n_mes),
    "ano": np.arange(n_ano_total)   # <-- AGORA TEM 4 ANOS (0,1,2,3)
}
if n_cov > 0:
    coords["covariate"] = np.arange(n_cov)

with pm.Model(coords=coords) as modelo:

    # Observados
    mes_idx_data = pm.Data("mes_idx", mes_idx, dims="obs_id")
    ano_idx_data = pm.Data("ano_idx", ano_idx, dims="obs_id")

    # Previsão
    mes_pred = pm.Data("mes_pred", np.zeros(12, dtype="int32"), dims="t_pred")
    ano_pred = pm.Data("ano_pred", np.zeros(12, dtype="int32"), dims="t_pred")

    if n_cov > 0:
        X_data = pm.Data("X", X_matrix, dims=("obs_id", "covariate"))
        X_pred = pm.Data("X_pred", np.zeros((12, n_cov)), dims=("t_pred","covariate"))
    else:
        X_pred = None

    # Priors (mesmas do treinamento)
    alpha0 = pm.Normal("alpha0", 9.8, 1.0)

    sigma_mes = pm.Exponential("sigma_mes", 2.0)
    mes_raw = pm.Normal("mes_raw", 0, 1, dims="mes")
    efeito_mes = pm.Deterministic("efeito_mes", mes_raw * sigma_mes, dims="mes")

    sigma_ano = pm.Exponential("sigma_ano", 2.0)
    ano_raw = pm.Normal("ano_raw", 0, 1, dims="ano")
    efeito_ano = pm.Deterministic("efeito_ano", ano_raw * sigma_ano, dims="ano")

    if n_cov > 0:
        beta = pm.Normal("beta", 0, 0.5, dims="covariate")
        cov_obs = pm.math.dot(X_data, beta)
        cov_pred = pm.math.dot(X_pred, beta)
    else:
        cov_obs = 0
        cov_pred = 0

    log_mu = alpha0 + efeito_mes[mes_idx_data] + efeito_ano[ano_idx_data] + cov_obs
    mu = pm.Deterministic("mu", pm.math.exp(log_mu), dims="obs_id")

    log_mu_pred = alpha0 + efeito_mes[mes_pred] + efeito_ano[ano_pred] + cov_pred
    mu_pred = pm.Deterministic("mu_pred", pm.math.exp(log_mu_pred), dims="t_pred")

    alpha_nb = pm.Exponential("alpha_nb", 1.0)

    y_obs = pm.NegativeBinomial("y_obs", mu=mu, alpha=alpha_nb, observed=y, dims="obs_id")

# -------------------------------------------------------
# Carregar idata
# -------------------------------------------------------

with open(f"{DATA_DIR}/idata_modelofinal.pkl", "rb") as f:
    idata = pickle.load(f)

# -------------------------------------------------------
# Previsão 2025
# -------------------------------------------------------

meses_2025 = np.arange(12).astype("int32")
ano_2025 = np.full(12, 3, dtype="int32")   # ano_idx=3 → válido agora

if n_cov > 0:
    X_2025 = np.zeros((12, n_cov))
else:
    X_2025 = None

with modelo:
    pm.set_data({"mes_pred": meses_2025, "ano_pred": ano_2025})
    if X_2025 is not None:
        pm.set_data({"X_pred": X_2025})

    ppc_2025 = pm.sample_posterior_predictive(
        idata,
        var_names=["mu_pred", "y_obs"],
        random_seed=123
    )

# -------------------------------------------------------
# Extrair resultados
# -------------------------------------------------------

y_samples = ppc_2025.posterior_predictive["y_obs"].values.reshape(-1, 12)
y_med = np.median(y_samples, axis=0)
y_hdi = az.hdi(y_samples, hdi_prob=0.95)

mes_nomes = [
    "JANEIRO","FEVEREIRO","MARÇO","ABRIL","MAIO","JUNHO",
    "JULHO","AGOSTO","SETEMBRO","OUTUBRO","NOVEMBRO","DEZEMBRO"
]

resultados = []
for i in range(12):
    resultados.append({
        "ano": 2025,
        "mes": mes_nomes[i],
        "mes_num": i + 1,
        "y_pred_mediana": float(y_med[i]),
        "y_pred_hdi_low": float(y_hdi[i, 0]),
        "y_pred_hdi_high": float(y_hdi[i, 1])
    })

with open(f"{DATA_DIR}/predicoes_2025.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=2)

print("\nOK! Previsão 2025 salva em data/bayes/modelofinal/predicoes_2025.json")


Sampling: [ano_raw, y_obs]


Output()


OK! Previsão 2025 salva em data/bayes/modelofinal/predicoes_2025.json


/tmp/ipykernel_53352/1604663553.py:157: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  y_hdi = az.hdi(y_samples, hdi_prob=0.95)


In [25]:
"""
Script: treinar_modelo_bayesiano_final.py

Modelo bayesiano NEGATIVE BINOMIAL hierárquico para previsão de
ocorrências atendidas pela PMDF (2022–2024).

Saídas salvas em:
    data/bayes/modelofinal/idata_modelofinal.pkl
    data/bayes/modelofinal/posterior_summary.json
    data/bayes/modelofinal/predicoes_in_sample.json
    data/bayes/modelofinal/model_config.json
"""

import os
import json
import pickle
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az


# -----------------------------------------------------------------------------
# 1. Carregamento e preparação dos dados
# -----------------------------------------------------------------------------

CSV_PATH = "data/PMDF_ocorrencias_2022-2024.csv"
OUTPUT_DIR = "data/bayes/modelofinal_2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)

# Mapeia mês em número
map_mes = {
    "JANEIRO": 1,
    "FEVEREIRO": 2,
    "MARÇO": 3,
    "MARCO": 3,
    "ABRIL": 4,
    "MAIO": 5,
    "JUNHO": 6,
    "JULHO": 7,
    "AGOSTO": 8,
    "SETEMBRO": 9,
    "OUTUBRO": 10,
    "NOVEMBRO": 11,
    "DEZEMBRO": 12,
}
df["mes_num"] = df["mes"].map(map_mes)

# Ordena temporalmente
df = df.sort_values(["ano", "mes_num"]).reset_index(drop=True)

# Índices hierárquicos
df["mes_idx"] = df["mes_num"] - 1
df["ano_idx"] = df["ano"].astype("category").cat.codes

y = df["ocor_atend"].astype("int64").values
n_obs = len(df)
n_mes = df["mes_idx"].nunique()
n_ano = df["ano_idx"].nunique()

mes_idx = df["mes_idx"].values
ano_idx = df["ano_idx"].values

# -----------------------------------------------------------------------------
# 2. Covariáveis (opcionais)
# -----------------------------------------------------------------------------

COVARIATE_CANDIDATES = [
    "arm_fogo_apr",
    "arm_branc_apr"
]

covariate_cols = [c for c in COVARIATE_CANDIDATES if c in df.columns]

if covariate_cols:
    X = df[covariate_cols].astype(float).copy()
    X_mean = X.mean()
    X_std = X.std().replace(0, 1.0)
    X_z = (X - X_mean) / X_std
    X_matrix = X_z.values
    n_cov = X_matrix.shape[1]
else:
    X_matrix = None
    n_cov = 0
    X_mean = pd.Series(dtype=float)
    X_std = pd.Series(dtype=float)

covariate_metadata = {
    "covariate_cols": covariate_cols,
    "means": X_mean.to_dict(),
    "stds": X_std.to_dict(),
}

# -----------------------------------------------------------------------------
# 3. Modelo Bayesiano final (NegBin hierárquico sem RW)
# -----------------------------------------------------------------------------

coords = {
    "obs_id": np.arange(n_obs),
    "mes": np.arange(n_mes),
    "ano": np.arange(n_ano),
}
if n_cov > 0:
    coords["covariate"] = np.arange(n_cov)

with pm.Model(coords=coords) as modelo_final:

    # Dados (pm.Data é mutável no PyMC 5.25.1)
    mes_idx_data = pm.Data("mes_idx", mes_idx, dims="obs_id")
    ano_idx_data = pm.Data("ano_idx", ano_idx, dims="obs_id")
    if n_cov > 0:
        X_data = pm.Data("X", X_matrix, dims=("obs_id", "covariate"))

    # Intercepto global (no log da média)
    # 9.5–10 é da ordem de grandeza das ocorrências (exp(9.5) ~ 13k, exp(10) ~ 22k)
    alpha0 = pm.Normal("alpha0", mu=9.8, sigma=1.0)

    # Efeito aleatório de mês (sazonalidade)
    sigma_mes = pm.Exponential("sigma_mes", 2.0)
    mes_raw = pm.Normal("mes_raw", 0.0, 1.0, dims="mes")
    efeito_mes = pm.Deterministic("efeito_mes", mes_raw * sigma_mes, dims="mes")

    # Efeito aleatório de ano
    sigma_ano = pm.Exponential("sigma_ano", 2.0)
    ano_raw = pm.Normal("ano_raw", 0.0, 1.0, dims="ano")
    efeito_ano = pm.Deterministic("efeito_ano", ano_raw * sigma_ano, dims="ano")

    # Covariáveis padronizadas
    if n_cov > 0:
        beta = pm.Normal("beta", mu=0.0, sigma=0.5, dims="covariate")
        cov_effect = pm.math.dot(X_data, beta)
    else:
        beta = None
        cov_effect = 0.0

    # Preditor linear
    log_mu = alpha0 + efeito_mes[mes_idx_data] + efeito_ano[ano_idx_data] + cov_effect
    mu = pm.Deterministic("mu", pm.math.exp(log_mu), dims="obs_id")

    # Overdispersion da NegBin
    alpha_nb = pm.Exponential("alpha_nb", 1.0)

    # Likelihood
    y_obs = pm.NegativeBinomial(
        "y_obs",
        mu=mu,
        alpha=alpha_nb,
        observed=y,
        dims="obs_id",
    )

    # Amostragem – target_accept alto para reduzir divergências
    idata = pm.sample(
        draws=2000,
        tune=2000,
        chains=4,
        cores=4,
        target_accept=0.98,
        max_treedepth=12,
        random_seed=123,
    )

    # Posterior preditiva in-sample
    ppc = pm.sample_posterior_predictive(
        idata,
        var_names=["y_obs", "mu"],
        random_seed=123,
    )

idata.extend(ppc)

# -----------------------------------------------------------------------------
# 4. Resumos e objetos para o Dashboard
# -----------------------------------------------------------------------------

var_names_summary = ["alpha0", "sigma_mes", "sigma_ano", "alpha_nb"]
if n_cov > 0:
    var_names_summary.append("beta")

summary_df = az.summary(idata, var_names=var_names_summary)
summary_json = summary_df.reset_index().to_dict(orient="records")

# Predições in-sample (mediana + IC 95%)
y_pred_samples = idata.posterior_predictive["y_obs"].values  # (chain, draw, obs)
y_pred_flat = y_pred_samples.reshape(-1, n_obs)

y_pred_mediana = np.median(y_pred_flat, axis=0)
y_pred_hdi = az.hdi(y_pred_flat, hdi_prob=0.95)  # (obs, 2)

pred_df = df[["ano", "mes", "mes_num", "ocor_atend"]].copy()
pred_df["y_pred_mediana"] = y_pred_mediana
pred_df["y_pred_hdi_low"] = y_pred_hdi[:, 0]
pred_df["y_pred_hdi_high"] = y_pred_hdi[:, 1]

predicoes_json = pred_df.to_dict(orient="records")

model_config = {
    "family": "NegativeBinomial",
    "link": "log",
    "formula": (
        "ocor_atend ~ 1 + (1 | mes) + (1 | ano)"
        + (" + covariaveis_padronizadas" if n_cov > 0 else "")
    ),
    "priors": {
        "alpha0": "Normal(9.8, 1.0)",
        "sigma_mes": "Exponential(2.0)",
        "sigma_ano": "Exponential(2.0)",
        "alpha_nb": "Exponential(1.0)",
        "beta": "Normal(0, 0.5) em covariáveis z-score" if n_cov > 0 else None,
    },
    "covariate_metadata": covariate_metadata,
    "descricao": (
        "Modelo bayesiano hierárquico NegBin com efeitos aleatórios de mês "
        "e ano, e covariáveis de apreensões padronizadas (quando presentes)."
    ),
}

# -----------------------------------------------------------------------------
# 5. Salvando .pkl e .json
# -----------------------------------------------------------------------------

idata_path = os.path.join(OUTPUT_DIR, "idata_modelofinal.pkl")
with open(idata_path, "wb") as f:
    pickle.dump(idata, f)

summary_path = os.path.join(OUTPUT_DIR, "posterior_summary.json")
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary_json, f, ensure_ascii=False, indent=2)

pred_path = os.path.join(OUTPUT_DIR, "predicoes_in_sample.json")
with open(pred_path, "w", encoding="utf-8") as f:
    json.dump(predicoes_json, f, ensure_ascii=False, indent=2)

config_path = os.path.join(OUTPUT_DIR, "model_config.json")
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(model_config, f, ensure_ascii=False, indent=2)

print("Treinamento concluído.")
print(f"InferenceData salvo em: {idata_path}")
print(f"Resumo posterior salvo em: {summary_path}")
print(f"Predições in-sample salvas em: {pred_path}")
print(f"Configuração do modelo salva em: {config_path}")

"""
Previsão 2025 – Versão Final Corrigida
"""

import os
import json
import pickle
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az


DATA_DIR = "data/bayes/modelofinal_2"
df = pd.read_csv("data/PMDF_ocorrencias_2022-2024.csv")

map_mes = {
    "JANEIRO": 1,"FEVEREIRO": 2,"MARÇO": 3,"MARCO": 3,"ABRIL": 4,"MAIO": 5,
    "JUNHO": 6,"JULHO": 7,"AGOSTO": 8,"SETEMBRO": 9,"OUTUBRO": 10,
    "NOVEMBRO": 11,"DEZEMBRO": 12
}

df["mes_num"] = df["mes"].map(map_mes)
df = df.sort_values(["ano", "mes_num"]).reset_index(drop=True)

df["mes_idx"] = df["mes_num"] - 1
df["ano_idx"] = df["ano"].astype("category").cat.codes

y = df["ocor_atend"].astype(int).values
mes_idx = df["mes_idx"].values.astype("int32")
ano_idx = df["ano_idx"].values.astype("int32")

n_obs = len(df)
n_mes = df["mes_idx"].nunique()

# ================================================================
# AQUI ESTÁ A CORREÇÃO DEFINITIVA:
# adicionar +1 ao número de anos para permitir ano_idx=3 (2025)
# ================================================================
n_ano_obs = df["ano_idx"].nunique()      # 3 anos observados
n_ano_total = n_ano_obs + 1              # adicionar ano 2025 → total=4

# -------------------------------------------------------
# Carregar metadata das covariáveis
# -------------------------------------------------------

with open(f"{DATA_DIR}/model_config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

cov_cols = config["covariate_metadata"]["covariate_cols"]
means = config["covariate_metadata"]["means"]
stds = config["covariate_metadata"]["stds"]

if cov_cols:
    X = df[cov_cols].copy().astype(float)
    X_z = (X - pd.Series(means)) / pd.Series(stds)
    X_matrix = X_z.values
    n_cov = X_matrix.shape[1]
else:
    X_matrix = None
    n_cov = 0

# -------------------------------------------------------
# Reconstruir o modelo com ano_total=4
# -------------------------------------------------------

coords = {
    "obs_id": np.arange(n_obs),
    "t_pred": np.arange(12),
    "mes": np.arange(n_mes),
    "ano": np.arange(n_ano_total)   # <-- AGORA TEM 4 ANOS (0,1,2,3)
}
if n_cov > 0:
    coords["covariate"] = np.arange(n_cov)

with pm.Model(coords=coords) as modelo:

    # Observados
    mes_idx_data = pm.Data("mes_idx", mes_idx, dims="obs_id")
    ano_idx_data = pm.Data("ano_idx", ano_idx, dims="obs_id")

    # Previsão
    mes_pred = pm.Data("mes_pred", np.zeros(12, dtype="int32"), dims="t_pred")
    ano_pred = pm.Data("ano_pred", np.zeros(12, dtype="int32"), dims="t_pred")

    if n_cov > 0:
        X_data = pm.Data("X", X_matrix, dims=("obs_id", "covariate"))
        X_pred = pm.Data("X_pred", np.zeros((12, n_cov)), dims=("t_pred","covariate"))
    else:
        X_pred = None

    # Priors (mesmas do treinamento)
    alpha0 = pm.Normal("alpha0", 9.8, 1.0)

    sigma_mes = pm.Exponential("sigma_mes", 2.0)
    mes_raw = pm.Normal("mes_raw", 0, 1, dims="mes")
    efeito_mes = pm.Deterministic("efeito_mes", mes_raw * sigma_mes, dims="mes")

    sigma_ano = pm.Exponential("sigma_ano", 2.0)
    ano_raw = pm.Normal("ano_raw", 0, 1, dims="ano")
    efeito_ano = pm.Deterministic("efeito_ano", ano_raw * sigma_ano, dims="ano")

    if n_cov > 0:
        beta = pm.Normal("beta", 0, 0.5, dims="covariate")
        cov_obs = pm.math.dot(X_data, beta)
        cov_pred = pm.math.dot(X_pred, beta)
    else:
        cov_obs = 0
        cov_pred = 0

    log_mu = alpha0 + efeito_mes[mes_idx_data] + efeito_ano[ano_idx_data] + cov_obs
    mu = pm.Deterministic("mu", pm.math.exp(log_mu), dims="obs_id")

    log_mu_pred = alpha0 + efeito_mes[mes_pred] + efeito_ano[ano_pred] + cov_pred
    mu_pred = pm.Deterministic("mu_pred", pm.math.exp(log_mu_pred), dims="t_pred")

    alpha_nb = pm.Exponential("alpha_nb", 1.0)

    y_obs = pm.NegativeBinomial("y_obs", mu=mu, alpha=alpha_nb, observed=y, dims="obs_id")

# -------------------------------------------------------
# Carregar idata
# -------------------------------------------------------

with open(f"{DATA_DIR}/idata_modelofinal.pkl", "rb") as f:
    idata = pickle.load(f)

# -------------------------------------------------------
# Previsão 2025
# -------------------------------------------------------

meses_2025 = np.arange(12).astype("int32")
ano_2025 = np.full(12, 3, dtype="int32")   # ano_idx=3 → válido agora

if n_cov > 0:
    X_2025 = np.zeros((12, n_cov))
else:
    X_2025 = None

with modelo:
    pm.set_data({"mes_pred": meses_2025, "ano_pred": ano_2025})
    if X_2025 is not None:
        pm.set_data({"X_pred": X_2025})

    ppc_2025 = pm.sample_posterior_predictive(
        idata,
        var_names=["mu_pred", "y_obs"],
        random_seed=123
    )

# -------------------------------------------------------
# Extrair resultados
# -------------------------------------------------------

y_samples = ppc_2025.posterior_predictive["y_obs"].values.reshape(-1, 12)
y_med = np.median(y_samples, axis=0)
y_hdi = az.hdi(y_samples, hdi_prob=0.95)

mes_nomes = [
    "JANEIRO","FEVEREIRO","MARÇO","ABRIL","MAIO","JUNHO",
    "JULHO","AGOSTO","SETEMBRO","OUTUBRO","NOVEMBRO","DEZEMBRO"
]

resultados = []
for i in range(12):
    resultados.append({
        "ano": 2025,
        "mes": mes_nomes[i],
        "mes_num": i + 1,
        "y_pred_mediana": float(y_med[i]),
        "y_pred_hdi_low": float(y_hdi[i, 0]),
        "y_pred_hdi_high": float(y_hdi[i, 1])
    })

with open(f"{DATA_DIR}/predicoes_2025.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=2)

print("\nOK! Previsão 2025 salva em data/bayes/modelofinal/predicoes_2025.json")


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha0, sigma_mes, mes_raw, sigma_ano, ano_raw, beta, alpha_nb]


Output()

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 25 seconds.
Sampling: [y_obs]


Output()

/tmp/ipykernel_53352/1470703593.py:190: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  y_pred_hdi = az.hdi(y_pred_flat, hdi_prob=0.95)  # (obs, 2)
Sampling: [ano_raw, y_obs]


Output()

Treinamento concluído.
InferenceData salvo em: data/bayes/modelofinal_2/idata_modelofinal.pkl
Resumo posterior salvo em: data/bayes/modelofinal_2/posterior_summary.json
Predições in-sample salvas em: data/bayes/modelofinal_2/predicoes_in_sample.json
Configuração do modelo salva em: data/bayes/modelofinal_2/model_config.json



OK! Previsão 2025 salva em data/bayes/modelofinal/predicoes_2025.json


/tmp/ipykernel_53352/1470703593.py:402: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  y_hdi = az.hdi(y_samples, hdi_prob=0.95)
